In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese_lccc/main')

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.2
GPU Enabled: False


In [4]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx


def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/lstm_seq2seq_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_export/1597745382/variables/variables


In [ ]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  for j in range(3):
    indices = [params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]
    print('Rank_{}:'.format(j+1), ' '.join(indices))
  print('%.2f sec' % (time.time() - t0))
  print()

Input:你好
Rank_1: 你 好
Rank_2: 我 好
Rank_3: 你 好 ！
0.92 sec

Input:早上好
Rank_1: 早 上 好
Rank_2: 早 安
Rank_3: 早 上 好 ！
0.05 sec

Input:晚上好
Rank_1: 晚 上 好
Rank_2: 晚 安
Rank_3: 早 上 好
0.06 sec

Input:再见
Rank_1: 再 见
Rank_2: 再 见 ！
Rank_3: 再 见 再 见
0.18 sec

Input:好久不见
Rank_1: 好 久 不 见
Rank_2: 是 啊
Rank_3: 是 的
0.10 sec

Input:想死你了
Rank_1: 我 也 想 你
Rank_2: 哈 哈
Rank_3: 我 也 是
0.19 sec

Input:谢谢你
Rank_1: 不 客 气
Rank_2: 客 气
Rank_3: 不 用 谢
0.07 sec

Input:爱你
Rank_1: 我 也 爱 你
Rank_2: 爱 你
Rank_3: 我 爱 你
0.09 sec

Input:你叫什么名字
Rank_1: 你 猜
Rank_2: 名 字
Rank_3: 什 么 名 字
0.19 sec

Input:你几岁了
Rank_1: 问 女 孩 子 年 龄 可 不 是 绅 士 所 为 哦 ~
Rank_2: 你 猜
Rank_3: 2 3
0.14 sec

Input:现在几点了
Rank_1: 不 知 道
Rank_2: 三 点
Rank_3: 十 一 点
0.19 sec

Input:今天天气怎么样
Rank_1: 天 气 怎 么 样
Rank_2: 天 气 很 好
Rank_3: 天 气 怎 么 样 ？
0.11 sec

Input:我们现在在哪里
Rank_1: 北 京
Rank_2: 在 家 里
Rank_3: 现 在 在 哪
0.20 sec

Input:你能给我讲个笑话吗
Rank_1: 不 能
Rank_2: 笑 话
Rank_3: 可 以
0.20 sec

Input:你是男孩还是女孩呀
Rank_1: 你 猜
Rank_2: 我 是
Rank_3: 我 是 女 孩
0.21 sec

Input:你会几种语言呀
Rank_1: 你 猜
Rank_2: 不